# Product vision and goals

In [103]:
#Vision
#An API that enables user to identify the most rar NFTs

In [112]:
#Goal 01: Create a trait_rarity_score

import pandas as pd
data = {'NFT_ID': [1322, 454, 1, 11],
        'Trait_ID': [1322, 454, 1, 11],
        'Type': ['Genesis', 'Genesis', 'none', 'Genesis'], 
        'Value': [0.5, 0.1, 0.5, 0.4],
        'Type_count': [123, 2, 0, 32], 
        'Type_rarity_percentage': [0.6, 0.6, 0.4, 0.4],
        'Trait_Rarity_Score': [16.6, 8.6, 8.6, 12.5],
        'Ranking_position': [16, 8, 8, 12],
       }

pd.DataFrame.from_dict(data)

,NFT_ID,Trait_ID,Type,Value,Type_count,Type_rarity_percentage,Trait_Rarity_Score,Ranking_position
0,1322,1322,Genesis,0.5,123,0.6,16.6,16
1,454,454,Genesis,0.1,2,0.6,8.6,8
2,1,1,none,0.5,0,0.4,8.6,8
3,11,11,Genesis,0.4,32,0.4,12.5,12


In [113]:
#Goal 02: Create a NFT_rarity_score 

import pandas as pd
data = {'NFT_ID': [1322, 454, 1, 11],
        'NFT_Rarity_Score': [16.6, 8.6, 8.6, 12.5],
        'NFT_Ranking_Position': [1 , 3, 3, 2]
        }

pd.DataFrame.from_dict(data)

,NFT_ID,NFT_Rarity_Score,NFT_Ranking_Position
0,1322,16.6,1
1,454,8.6,3
2,1,8.6,3
3,11,12.5,2


#  Imported moduls

In [111]:
#Import moduls
import requests
import collections
import pandas as pd
from itertools import islice


# Get Data via API

In [2]:
#Call OpenSea API to get CryptoTrunks Data
response = requests.get("https://api.opensea.io/api/v1/assets?collection=cryptotrunks&format=json&limit=50&offset=50&order_direction=desc")
print(response.status_code)

200


In [15]:
#Save first 50 CryptoTrunks Data as JSON
Crypto_trunks_50_json = response.json()
Crypto_trunks_50_json

{'assets': [{'id': 37037838,
   'token_id': '210',
   'num_sales': 0,
   'background_color': None,
   'image_url': 'https://lh3.googleusercontent.com/rcyl4LohJGUpGdXcaUyevEtEvm9-lceiH3JrVYPdbu2W_K8o6ylLKE6Cr2XKNK59AauIpweOGdePK-cAR9mSoWh9KxspgYMQ1SbuKw',
   'image_preview_url': 'https://lh3.googleusercontent.com/rcyl4LohJGUpGdXcaUyevEtEvm9-lceiH3JrVYPdbu2W_K8o6ylLKE6Cr2XKNK59AauIpweOGdePK-cAR9mSoWh9KxspgYMQ1SbuKw=s250',
   'image_thumbnail_url': 'https://lh3.googleusercontent.com/rcyl4LohJGUpGdXcaUyevEtEvm9-lceiH3JrVYPdbu2W_K8o6ylLKE6Cr2XKNK59AauIpweOGdePK-cAR9mSoWh9KxspgYMQ1SbuKw=s128',
   'image_original_url': 'https://service.cryptotrunks.co/image.png?token=210',
   'animation_url': None,
   'animation_original_url': None,
   'name': 'CryptoTrunk #210',
   'description': None,
   'external_link': 'https://cryptotrunks.co/individual-trunk-page.html?token=210',
   'asset_contract': {'address': '0x375ea781c49eafedde07afe6196f885761f166ae',
    'asset_contract_type': 'non-fungible',
   

# Data preprocessing

In [70]:
#List of NFT_IDs
List_of_50_NFTs = Crypto_trunks_50_json['assets']
NFT_ID =[List_of_50_NFTs[NFT]['token_id'] 
          for NFT in range(len(Crypto_trunks_50_json['assets']))]

print(len(NFT_ID))
print(NFT_ID[0:5])

50
['210', '733', '1256', '279', '802']


In [71]:
#Trait_types in JSON
List_of_Traits_in_all_NFTs = [
    List_of_50_NFTs[NFT]['traits'] 
    for NFT in range(len(Crypto_trunks_50_json['assets']))]
List_of_Traits_in_all_NFTs

print(len(List_of_Traits_in_all_NFTs))
print(List_of_Traits_in_all_NFTs[0])

50
[{'trait_type': 'Type', 'value': 'Burning Ash', 'display_type': None, 'max_value': None, 'trait_count': 13, 'order': None}, {'trait_type': 'Age', 'value': 'Elder', 'display_type': None, 'max_value': None, 'trait_count': 2758, 'order': None}, {'trait_type': 'Edition', 'value': 'Genesis', 'display_type': None, 'max_value': None, 'trait_count': 1501, 'order': None}, {'trait_type': 'Element', 'value': 'Among Us – Orange', 'display_type': None, 'max_value': None, 'trait_count': 11, 'order': None}, {'trait_type': 'Sky', 'value': 'Underwater City and Road', 'display_type': None, 'max_value': None, 'trait_count': 21, 'order': None}]


# Goal 01:  Traits_Rarity_Score 

1. Create Lists for the DataFrame

In [72]:
# Trait_types
Trait_types_for_NFTs = [
    List_of_Traits_in_all_NFTs[NFT][TRAITE]['trait_type'] 
    for NFT in range(len(List_of_Traits_in_all_NFTs))
    for TRAITE in range(len(List_of_Traits_in_all_NFTs[NFT]))
    ]
Trait_types_for_NFTs[:5]

['Type', 'Age', 'Edition', 'Element', 'Sky']

In [110]:
#Trait_Values
#How to generate a value for every trait?
Trait_value_for_NFTs = [
    List_of_Traits_in_all_NFTs[NFT][TRAITE]['value'] 
    for NFT in range(len(List_of_Traits_in_all_NFTs))
    for TRAITE in range(len(List_of_Traits_in_all_NFTs[NFT]))
    ]

print(len(Trait_value_for_NFTs))
print(Trait_value_for_NFTs[:5])

301
['Burning Ash', 'Elder', 'Genesis', 'Among Us – Orange', 'Underwater City and Road']


In [74]:
#Trait_counts
Trait_count_for_NFTs = [
    List_of_Traits_in_all_NFTs[NFT][TRAITE]['trait_count'] 
    for NFT in range(len(List_of_Traits_in_all_NFTs))
    for TRAITE in range(len(List_of_Traits_in_all_NFTs[NFT]))
    ]

print(len(Trait_count_for_NFTs))
print(Trait_count_for_NFTs[0:5])

301
[13, 2758, 1501, 11, 21]


In [75]:
#Percentages%
#Trait_type_rarity_percentages

Total_amount_of_crypto_trunks = 23.511

Trait_rarity_percentage = [
    List_of_Traits_in_all_NFTs[NFT][TRAITE]['trait_count']/Total_amount_of_crypto_trunks
    for NFT in range(len(List_of_Traits_in_all_NFTs))
    for TRAITE in range(len(List_of_Traits_in_all_NFTs[NFT]))
    ]

print(len(Trait_rarity_percentage))
print(Trait_rarity_percentage[0:5])

301
[0.5529326698141296, 117.30679256518226, 63.84245672238527, 0.4678661052273404, 0.8931989281612862]


In [76]:
#NFT_IDs
# How do I create a DataFrame where all NFT_IDs match with amount of Traits?

NFT_IDs_for_each_trait = [
    List_of_50_NFTs[NFT]['token_id']
    for NFT in range(len(List_of_Traits_in_all_NFTs))
    for TRAITE in range(len(List_of_Traits_in_all_NFTs[NFT]))
    ]

print(len(NFT_IDs_for_each_trait))
print(NFT_IDs_for_each_trait[0:10])

301
['210', '210', '210', '210', '210', '733', '733', '733', '733', '733']


In [77]:
#Trait_rarity_score
# How to calculate a rarity_score for every trait?
Trait_rarity_score = [
    1/(List_of_Traits_in_all_NFTs[NFT][TRAITE]['trait_count']/Total_amount_of_crypto_trunks)
    for NFT in range(len(List_of_Traits_in_all_NFTs))
    for TRAITE in range(len(List_of_Traits_in_all_NFTs[NFT]))
    ]

print(len(Trait_rarity_score))
print(Trait_rarity_score[0:4])

301
[1.8085384615384614, 0.008524655547498186, 0.015663557628247834, 2.137363636363636]


2. Create DataSet and display it in a DataFrame

In [78]:
Total_amount_of_crypto_trunks = 23511 

data = {'NFT_IDs': NFT_IDs_for_each_trait, #Check
        'Trait_Types': Trait_types_for_NFTs, #Check 
        'Trait_Values': Trait_value_for_NFTs,#Check
        'Trait_Counts': Trait_count_for_NFTs, #Check
        'Type_Rarity_Percentages': Trait_rarity_percentage, #Check
        'Trait_Rarity_Scores': Trait_rarity_score
        #'Trait_IDs': [1,2,3,4,5,1,2,3,1,2,3,4,] #Do it with pandas
        #'Ranking_position': [16.6, 8.6, 8.6, 12.5], #Do it with pandas
       }

df = pd.DataFrame.from_dict(data)
df.head()

,NFT_IDs,Trait_Types,Trait_Values,Trait_Counts,Type_Rarity_Percentages,Trait_Rarity_Scores
0,210,Type,Burning Ash,13,0.552933,1.808538
1,210,Age,Elder,2758,117.306793,0.008525
2,210,Edition,Genesis,1501,63.842457,0.015664
3,210,Element,Among Us – Orange,11,0.467866,2.137364
4,210,Sky,Underwater City and Road,21,0.893199,1.119571


In [79]:
#Trait_IDs
df['Trait_Ids'] = df.groupby('NFT_IDs').cumcount() + 1
df

,NFT_IDs,Trait_Types,Trait_Values,Trait_Counts,Type_Rarity_Percentages,Trait_Rarity_Scores,Trait_Ids
0,210,Type,Burning Ash,13,0.552933,1.808538,1
1,210,Age,Elder,2758,117.306793,0.008525,2
2,210,Edition,Genesis,1501,63.842457,0.015664,3
3,210,Element,Among Us – Orange,11,0.467866,2.137364,4
4,210,Sky,Underwater City and Road,21,0.893199,1.119571,5
...,...,...,...,...,...,...,...
296,337,Type,Birch Treehouse,48,2.041598,0.489812,3
297,337,Background,Grass – Soft,88,3.742929,0.267170,4
298,337,Edition,Genesis,1501,63.842457,0.015664,5
299,337,Age,Adult,4225,179.703118,0.005565,6


In [80]:
#Sort columns
col = df.pop("Trait_Ids")
df.insert(1, col.name, col)
df

,NFT_IDs,Trait_Ids,Trait_Types,Trait_Values,Trait_Counts,Type_Rarity_Percentages,Trait_Rarity_Scores
0,210,1,Type,Burning Ash,13,0.552933,1.808538
1,210,2,Age,Elder,2758,117.306793,0.008525
2,210,3,Edition,Genesis,1501,63.842457,0.015664
3,210,4,Element,Among Us – Orange,11,0.467866,2.137364
4,210,5,Sky,Underwater City and Road,21,0.893199,1.119571
...,...,...,...,...,...,...,...
296,337,3,Type,Birch Treehouse,48,2.041598,0.489812
297,337,4,Background,Grass – Soft,88,3.742929,0.267170
298,337,5,Edition,Genesis,1501,63.842457,0.015664
299,337,6,Age,Adult,4225,179.703118,0.005565


# Goal 02: NFT_rarity_score

In [81]:
#NFT_rarity_Score

NFT_rarity_score = []

count_of_traits_per_NFT = collections.Counter(NFT_IDs_for_each_trait)
count_of_traits_per_NFT

print(len(count_of_traits_per_NFT))
print(count_of_traits_per_NFT)

50
Counter({'802': 9, '969': 9, '1463': 8, '486': 8, '693': 8, '1285': 8, '733': 7, '1256': 7, '555': 7, '1147': 7, '1216': 7, '239': 7, '308': 7, '1492': 7, '1038': 7, '1107': 7, '130': 7, '653': 7, '337': 7, '279': 6, '1325': 6, '940': 6, '101': 6, '624': 6, '900': 6, '446': 6, '1176': 6, '791': 6, '1314': 6, '210': 5, '348': 5, '871': 5, '1394': 5, '417': 5, '1009': 5, '32': 5, '762': 5, '831': 5, '1423': 5, '61': 5, '584': 5, '199': 5, '722': 5, '1245': 5, '268': 5, '1078': 4, '170': 4, '1354': 4, '377': 4, '515': 4})


In [82]:
length_per_trait = list(count_of_traits_per_NFT.values())
print(length_per_trait)

[5, 7, 7, 6, 9, 6, 5, 5, 5, 5, 6, 8, 8, 5, 5, 7, 4, 6, 6, 7, 4, 8, 7, 7, 5, 8, 7, 5, 4, 4, 6, 5, 6, 9, 7, 4, 7, 5, 5, 7, 7, 7, 6, 5, 5, 5, 5, 6, 6, 7]


In [86]:
#NFT_rarity_score
# Input list initialization
Input = Trait_rarity_score
  
# list of length in which we have to split
length_to_split = length_per_trait
  
# Using islice
Inputt = iter(Input)
NFT_rarity_score = [sum(list(islice(Inputt, elem)))
          for elem in length_to_split]
  
# Printing Output
#print(Input)
#print("Split length list: ", length_to_split)
print(len(NFT_rarity_score))
print(NFT_rarity_score[:5])

50
[5.089661739649272, 1.6522857502913788, 3.806742255728938, 0.7841268831946387, 1.4790641477063755]


In [115]:
data = {'NFT_ID': NFT_ID,
        'NFT_Rarity_Score': NFT_rarity_score
        }

df = pd.DataFrame.from_dict(data).sort_values(by='NFT_Rarity_Score', ascending=False)
df.head()

,NFT_ID,NFT_Rarity_Score
0,210,5.089662
29,377,3.989950
17,101,3.979075
2,1256,3.806742
43,199,3.530149


In [96]:
# Creating a new column with a rank
df['NFT_Ranking_Position'] = df['NFT_Rarity_Score'].rank(ascending=False)

In [101]:
Ranked_table = df.set_index('NFT_Ranking_Position')
Ranked_table.head()

,NFT_ID,NFT_Rarity_Score
NFT_Ranking_Position,,
1.0,210,5.089662
2.0,377,3.989950
3.0,101,3.979075
4.0,1256,3.806742
5.0,199,3.530149
